# Automate scenario runs

The process involves:
1. running a scenario-specific spinup, initialized with `spin3.out.press.pfb` using `scen_spin.tcl` (this will simulate 5 years of 2010 meteorological forcing
2. Running the variable dz subsurface post processing script (python with fortran module, which needs to be installed using: `f2py -c -m postproc scale_pfb.f pfb_read.f writepfb.f` and has dependencies: `scale_pfb.f pfb_read.f writepfb.f`) Need to confirm that subsurface storage is changing by less than 1% between years 4 and 5.
3. Use the last pfb file `slopes_only.out.press.43805.pfb` as the input `spin4.out.press.pfb` (save this in each scen's directory) to the one year simulation
4. Remove all spinup run files to make space
5. Run the 1-year simulation using `scen_run.tcl`
6. Run the full water balance script using `postproc` f2py module. Save the csv dataframe in outputs directory.
7. Remove all run files.

In [23]:
import subprocess
import glob
import os
import shutil
import time

from postproc import scale_pfb
import pandas as pd
from io import StringIO
import itertools
import numpy as np
from shlex import split

%matplotlib inline

In [19]:
# the directory that contains all the subdirectories for each scenario
#scendir_base = '/home/tclim/Dropbox/projects/pf_simple/00preproc/01subsurf_scen'
scendir_base = '/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf'
clm_inputs_dir = '/home/theo/pf_files/pf_machine/scenarios/scens_inputs_clm'

# Copy scenario subsurface inputs into run directory

In [17]:
scen = 'scen000'

In [8]:
[print (f) for f in glob.glob('%s/%s/*' %(scendir_base, scen))]


/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf/scen000/mann_scen.pfb
/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf/scen000/perm_scen.pfb
/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf/scen000/ind_scen.pfb


[None, None, None]

In [9]:
'%s/%s/*' %(scendir_base, scen)

'/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf/scen000/*'

In [19]:
shutil.copy2('%s/%s/mann_scen.pfb' %(scendir_base, scen), '.')
shutil.copy2('%s/%s/ind_scen.pfb' %(scendir_base, scen), '.')
shutil.copy2('%s/%s/perm_scen.pfb' %(scendir_base, scen), '.')

'./perm_scen.pfb'

## Copy CLM files into run directory 

In [20]:
shutil.copy2('%s/dauphco.nldas.10yr.txt' %clm_inputs_dir, '.')
shutil.copy2('%s/drv_clmin.dat' %clm_inputs_dir, '.')
shutil.copy2('%s/drv_vegm.dat' %clm_inputs_dir, '.')
shutil.copy2('%s/drv_vegp.dat' %clm_inputs_dir, '.')  ## HAVE TO CHANGE INPUTS!!

'./drv_vegp.dat'

# Run Scenario-specific spinup

In [21]:
start = time.time()
bashCommand = "tclsh scen_spin.tcl"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Time Elapsed:  3505.763441324234 s


In [22]:
print(output.decode("utf-8").format())






ParFlow run complete
Using process grid (1,1,1)
Using process grid (1,1,1)
Using process grid (1,1,1)
Using process grid (1,1,1)



# Calculate vdz_subsurf storage

In [5]:
rundir = '/home/theo/pf_files/pf_machine/scenarios/rundir'
cur_stop = 43805 # five years of scenario-specific spinup

In [8]:
# create silos needed for subsurf stor
bashCommand = "tclsh write_subsurf_silos.tcl %s %s %s %s" %(rundir,'slopes_only',0,cur_stop)
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print(output)
print(error)

b''
None


In [6]:
def silo2pfb(rundir, bnam, start, stop, fw=0):
    '''
    Converts a timeseries of silo output to pfb format
    (saves converted pfbs to same directory)
    
    bnam     the basename of the files (everything up to the iterating index)
    start    start number of file indexing
    end      end number of file indexing
    fw       fixed width = 0 for non-fixed width iterating index, 
             fixed width = 1 for fixed width (assumed width is 5) index.
    '''
    bashCommand = "tclsh silotopfb_iter_bnam.tcl %s %s %s %s %s" %(rundir, bnam,start,stop, fw)
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    print(output)
    print(error)
    
    # if fw =1, rename the output files to remove $runname.out
    if fw:
        print("renaming output pfb files")
        newbnam = bnam.split(".")[-1]
        
        for i in range(start, stop+1):
            ofnam = "%s/%s.%s.pfb" %(rundir,bnam,i)
            nfnam = "%s/%s.%s.pfb" %(rundir,newbnam,i)
            os.rename(ofnam, nfnam)

def sumoverdomain(rundir, bnam, start, stop):
    '''
    Sums over the entire domain for an input pfb file,
    returns a dataframe with columns for t and the sum values
    
    bnam    the base name of the files to sum
    start   the start number of file indexing
    end     the end number of file indexing
    '''
    bashCommand = "tclsh sum_domain_bnam.tcl %s %s %s %s" %(rundir, bnam,start,stop)
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

    # Save the output to a dataframe
    output = output.decode("utf-8").format()
    df = pd.read_csv(StringIO(output), sep="\t", header = None)
    df.columns = ['t','sum_val']  # add a header (column name)
    
    # return
    return(df)

In [7]:
factors = np.array([[2.0],  # <- bottom
                    [2.0],
                    [2.0],
                    [1.0],
                    [1.0],
                    [1.0],
                    [0.25],
                    [0.25],
                    [0.25],
                    [0.25]]) # <- top

In [11]:

# convert silos to pfbs
silo2pfb(rundir = rundir, bnam ='subsurface_storage' , start= 0, stop=cur_stop)




b''
None


In [8]:
# scale pfbs using factors
for i in range(0,cur_stop+1):
    infnam = '%s/subsurface_storage.%s.pfb' %(rundir, i)
    outfnam = '%s/vdz_subsurface_storage.%s.pfb' %(rundir,i)
    
    scale_pfb(pfbinfnam = infnam, vdzarr = factors, pfboutfnam = outfnam, 
         nx = 12, ny = 10, dx = 10, dy = 10, dz = 1)
    
ss = sumoverdomain(rundir = rundir,
                   bnam = 'vdz_subsurface_storage',start = 0, stop= cur_stop)
ss.head(10)


,t,sum_val
0,0,3.261884e+06
1,1,3.262281e+06
2,2,3.262579e+06
3,3,3.262966e+06
4,4,3.263003e+06
5,5,3.263001e+06
6,6,3.262908e+06
7,7,3.262781e+06
8,8,3.262654e+06
9,9,3.262527e+06


### Check each year's volume pc change

In [15]:
ss['sum_val'][list(np.multiply(8610, [0,1,2,3,4,5]))].pct_change()

0             NaN
8610     0.010764
17220    0.012395
25830    0.007813
34440    0.002453
43050   -0.004638
Name: sum_val, dtype: float64

# Save and rename the last pressure file
Put it in the scenario directory

In [20]:
'%s/%s/spin4.out.press.pfb' %(scendir_base, scen)

'/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf/scen000/mann_scen.pfb'

In [21]:
os.rename('slopes_only.out.press.43805.pfb', '%s/%s/spin4.out.press.pfb' %(scendir_base, scen))

# Remove memory-eating runtime files

This is the bash script:

```
find . -name "slopes_only*" -print0 | xargs -0 rm
find . -name "*silo" -print0 | xargs -0 rm
find . -name "*pfb" -print0 | xargs -0 rm
rm gp*
```

In [25]:
start = time.time()
p1 = subprocess.Popen(split('find . -name "slopes_only*" -print0'), stdout=subprocess.PIPE)
p2 = subprocess.Popen(split('xargs -0 rm -rf'), stdin=p1.stdout)
print('Removed slopes_only*')
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Removed slopes_only*
Time Elapsed:  0.02194666862487793 s


In [29]:
start = time.time()
p1 = subprocess.Popen(split('find . -name  "*silo"  -print0'), stdout=subprocess.PIPE)
p2 = subprocess.Popen(split('xargs -0 rm -rf'), stdin=p1.stdout)
print('Removed *silo')
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Removed *silo
Time Elapsed:  0.022481918334960938 s


In [30]:
start = time.time()
p1 = subprocess.Popen(split('find . -name  "*pfb"  -print0'), stdout=subprocess.PIPE)
p2 = subprocess.Popen(split('xargs -0 rm -rf'), stdin=p1.stdout)
print('Removed *pfb')
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Removed *pfb
Time Elapsed:  0.022441387176513672 s


In [31]:
start = time.time()
p1 = subprocess.Popen(split('find . -name  "gp*"  -print0'), stdout=subprocess.PIPE)
p2 = subprocess.Popen(split('xargs -0 rm -rf'), stdin=p1.stdout)
print('Removed gp*')
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Removed gp*
Time Elapsed:  0.02640056610107422 s


# Re-copy input files scenario directory

This should include the original subsurface pfbs, and spin4.out.press.pfb from the scenario-specific spinup

In [41]:
scendir_base
scen
scendir = '%s/%s' %(scendir_base, scen)
scendir

'/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf/scen000'

In [43]:
scen_files

['mann_scen.pfb', 'spin4.out.press.pfb', 'perm_scen.pfb', 'ind_scen.pfb']

In [44]:
scen_files = os.listdir(scendir)
for file_name in scen_files:
    full_file_name = os.path.join(scendir, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, '.')

# Run one year

In [48]:
start = time.time()
bashCommand = "tclsh scen_run.tcl"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Time Elapsed:  705.4529099464417 s


In [47]:
print(output.decode("utf-8").format())

# Calculate Water Balance

## Save wb csv

# Remove run files